In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

#### Train an example model

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel

##### All the parameters are in the param file, you can look at them if you want

In [4]:
param_file = '../../cellbgnet/utils/reference_files/reference_venus.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [5]:
param

namespace(Simulation=namespace(train_size=128,
                               train_type='cells',
                               psf_intensity_normalization=True,
                               density=0.003125,
                               non_cell_density=10,
                               margin_empty=0.05,
                               z_prior=[-1, 1],
                               survival_prob=None,
                               min_photon=0.25,
                               camera='sCMOS',
                               camera_chip_size=[1041, 1302],
                               perlin_noise=False,
                               perlin_noise_factor=0.2,
                               perlin_noise_res=64,
                               baseline=103.0,
                               bg_values=115.0,
                               bg_width=2,
                               e_per_adu=0.38,
                               qe=0.95,
                               spurious_c=0.0,

In [6]:
model = cellbgnet.model.CellBGModel(param)


training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21, field_xy: [1026, 1153, 114, 24

In [7]:
model.init_recorder()
model.init_eval_data()

Eval image generation:   0%|                                                                | 0/30 [00:00<?, ?it/s]/home/pk/Documents/cellbgnet/cellbgnet/utils/hardware.py:9: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:83.)
  return torch.cuda.FloatTensor(x.astype('float32'))
Eval image generation: 100%|███████████████████████████████████████████████████████| 30/30 [00:10<00:00,  2.96it/s]


eval images shape: (30, 1041, 1302) contain 13601 molecules,


In [8]:
start_time = time.time()
model.fit(batch_size=32, max_iters=20000, print_output=True, print_freq=100)
duration = time.time() - start_time

Started training .... 
 || Cost: 6301.770 || Time Upd.: 388.6 ms  || BatchNr.: 100
 || Cost: 211.434 || Time Upd.: 379.9 ms  || BatchNr.: 200
 || Cost: 3109.616 || Time Upd.: 383.8 ms  || BatchNr.: 300
 || Cost: 68.777 || Time Upd.: 382.0 ms  || BatchNr.: 400
 || Cost: 76.665 || Time Upd.: 385.9 ms  || BatchNr.: 500
 || Cost: 47.501 || Time Upd.: 386.6 ms  || BatchNr.: 600
 || Cost: 40.125 || Time Upd.: 381.9 ms  || BatchNr.: 700
 || Cost: 43.309 || Time Upd.: 380.5 ms  || BatchNr.: 800
 || Cost: 34.152 || Time Upd.: 398.0 ms  || BatchNr.: 900
 || Cost: 31.467 || Time Upd.: 418.0 ms  || BatchNr.: 1000
processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 13601FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 13599 ,preds: 10317
JoR: 2.008 || Eff_3d: 46.155 || Jaccard: 0.757 || Factor: 371.772 || RMSE_lat: 37.691 || RMSE_ax: 115.909 || RMSE_x: 26.